In [6]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

In [7]:
violations_file = "csvFiles/HealthViolations.csv"
violations_df = pd.read_csv(violations_file)
violations_df.head()

,serial_number,activity_date,facility_name,violation_code,violation_description,violation_status,points,grade,facility_address,facility_city,...,owner_name,pe_description,program_element_pe,program_name,program_status,record_id,score,service_code,service_description,row_id
0,DA08R0TCU,2018-03-30T00:00:00,KRUANG TEDD,F030,# 30. Food properly stored; food storage conta...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF030
1,DA08R0TCU,2018-03-30T00:00:00,KRUANG TEDD,F027,# 27. Food separated and protected,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF027
2,DA08R0TCU,2018-03-30T00:00:00,KRUANG TEDD,F035,# 35. Equipment/Utensils - approved; installed...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF035
3,DA08R0TCU,2018-03-30T00:00:00,KRUANG TEDD,F033,# 33. Nonfood-contact surfaces clean and in go...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF033
4,DA08R0TCU,2018-03-30T00:00:00,KRUANG TEDD,F029,"# 29. Toxic substances properly identified, st...",OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,LOS ANGELES,...,5151 HOLLYWOOD LLC,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF029


In [8]:
violations_df.drop_duplicates("serial_number", inplace=True)

violations_df = violations_df.reset_index()

violations_df["id"] = violations_df.index

violations_df.head()

,index,serial_number,activity_date,facility_name,violation_code,violation_description,violation_status,points,grade,facility_address,...,pe_description,program_element_pe,program_name,program_status,record_id,score,service_code,service_description,row_id,id
0,0,DA08R0TCU,2018-03-30T00:00:00,KRUANG TEDD,F030,# 30. Food properly stored; food storage conta...,OUT OF COMPLIANCE,1,A,5151 HOLLYWOOD BLVD,...,RESTAURANT (31-60) SEATS HIGH RISK,1635,KRUANG TEDD,ACTIVE,PR0031205,92,1,ROUTINE INSPECTION,DA08R0TCUF030,0
1,7,DA0GBKW3F,2018-03-30T00:00:00,SPROUTS FARMERS MARKET#403,F044,"# 44. Floors, walls and ceilings: properly bui...",OUT OF COMPLIANCE,1,A,2245 YOSEMITE DR,...,"FOOD MKT RETAIL (25-1,999 SF) HIGH RISK",1612,MEAT MARKET,ACTIVE,PR0179289,97,1,ROUTINE INSPECTION,DA0GBKW3FF044,1
2,10,DA0HEAJNR,2018-03-30T00:00:00,LOS ANGELES NEW YORK PIZZA,F044,"# 44. Floors, walls and ceilings: properly bui...",OUT OF COMPLIANCE,1,A,518 W 6TH ST,...,RESTAURANT (31-60) SEATS MODERATE RISK,1634,LOS ANGELES NEW YORK PIZZA,ACTIVE,PR0031535,92,1,ROUTINE INSPECTION,DA0HEAJNRF044,2
3,15,DA0RZDWLN,2018-03-30T00:00:00,THE BAGEL FACTORY,F029,"# 29. Toxic substances properly identified, st...",OUT OF COMPLIANCE,1,A,8984 CADILLAC AVE,...,RESTAURANT (0-30) SEATS MODERATE RISK,1631,THE BAGEL FACTORY,ACTIVE,PR0037137,93,1,ROUTINE INSPECTION,DA0RZDWLNF029,3
4,22,DA0Z5W4J0,2018-03-30T00:00:00,CREPE EXPRESS,F014,# 14. Food contact surfaces: clean and sanitized,OUT OF COMPLIANCE,2,A,529 W 6TH ST,...,RESTAURANT (0-30) SEATS MODERATE RISK,1631,CREPE EXPRESS,ACTIVE,PR0037962,92,1,ROUTINE INSPECTION,DA0Z5W4J0F014,4


In [9]:
violations_df = violations_df[["id", "serial_number", "activity_date", "facility_name", "score", "violation_description"]]

violations_df.set_index("id", inplace=True)

violations_df.head(2)

,serial_number,activity_date,facility_name,score,violation_description
id,,,,,
0,DA08R0TCU,2018-03-30T00:00:00,KRUANG TEDD,92,# 30. Food properly stored; food storage conta...
1,DA0GBKW3F,2018-03-30T00:00:00,SPROUTS FARMERS MARKET#403,97,"# 44. Floors, walls and ceilings: properly bui..."


In [10]:
#violations_grouped = violations_df.groupby(["serial_number"])

#violations_count = violations_grouped.count()
#violations_count["violations_count"] = violations_count["facility_name"]
#violations_count["avg_score"]  = violations_grouped["score"].mean()


#violations_count = violations_count.sort_values("violations_count", ascending=False)

#violations_count = violations_count[["violations_count", "avg_score"]]

#violations_count

In [11]:
import json
data = json.load(open('jsonFiles/Inspections.json'))

inspections_df = pd.DataFrame(data["data"])
inspections_df.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,412181,332032D9-E7BD-4F8D-8C92-58CF5057A994,412181,1541201351,969808,1541201351,969808,None,2016-06-22T00:00:00,OW0185247,...,954 E ALOSTA AVE,AZUSA,CA,91702,001,ROUTINE INSPECTION,93,A,DAC6YRJ7D,EE0000742
1,428600,C78CB01A-453A-4E26-A21A-3FF79F6FBA83,428600,1541201357,969808,1541201357,969808,None,2016-09-07T00:00:00,OW0235635,...,7275 MELROSE AVE,LOS ANGELES,CA,90046,001,ROUTINE INSPECTION,90,A,DAKU06F5Y,EE0000951
2,429129,BF888E62-D91E-42AF-BB5B-1B9EBBEEC00B,429129,1541201357,969808,1541201357,969808,None,2016-09-09T00:00:00,OW0018013,...,28788 THE OLD RD,VALENCIA,CA,91355,001,ROUTINE INSPECTION,98,A,DAFS03QYK,EE0000735
3,392634,23EAC538-221D-450C-A2DA-E941E12D840F,392634,1541201336,969808,1541201336,969808,None,2016-04-01T00:00:00,OW0001761,...,16550 SOLEDAD CANYON RD,CANYON COUNTRY,CA,91387-3215,001,ROUTINE INSPECTION,99,A,DAJ8A0PDB,EE0000735
4,392635,945EE22C-428A-416B-8805-CF798AF24CED,392635,1541201336,969808,1541201336,969808,None,2016-04-01T00:00:00,OW0002023,...,2355 PECK RD,SOUTH EL MONTE,CA,91733,001,ROUTINE INSPECTION,92,A,DA6E75RM0,EE0000842


In [12]:
rename_inspections_df = inspections_df.rename(columns={8: "activity_date", 10: "owner_name", 21: "zipcode", 11: "facility_id", 13: "record_id", 18: "address", 19: "city", 20: "state", 23: "service_description", 24: "score", 25: "grade", 26: "serial_number", 9: "owner_id", 12: "facility_name"})

rename_inspections_df["id"]=rename_inspections_df.index
rename_inspections_df.reset_index()

rename_inspections_df.head(2)

,0,1,2,3,4,5,6,7,activity_date,owner_id,...,city,state,zipcode,22,service_description,score,grade,serial_number,27,id
0,412181,332032D9-E7BD-4F8D-8C92-58CF5057A994,412181,1541201351,969808,1541201351,969808,None,2016-06-22T00:00:00,OW0185247,...,AZUSA,CA,91702,001,ROUTINE INSPECTION,93,A,DAC6YRJ7D,EE0000742,0
1,428600,C78CB01A-453A-4E26-A21A-3FF79F6FBA83,428600,1541201357,969808,1541201357,969808,None,2016-09-07T00:00:00,OW0235635,...,LOS ANGELES,CA,90046,001,ROUTINE INSPECTION,90,A,DAKU06F5Y,EE0000951,1


In [13]:
inspections_columns = ["id", "serial_number", "service_description"]
inspections_df = rename_inspections_df[inspections_columns].copy()

inspections_df.set_index("id", inplace=True)

inspections_df.head(2)
                    

,serial_number,service_description
id,,
0,DAC6YRJ7D,ROUTINE INSPECTION
1,DAKU06F5Y,ROUTINE INSPECTION


In [16]:
connection_string = "root:Mario#5@localhost/restaurant_db"
engine = create_engine(f'mysql://{connection_string}', pool_recycle=3600)

In [17]:
# Confirm tables
engine.table_names()

C:\Users\JB52612\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


['inspections', 'violations']

In [18]:
inspections_df.to_sql(name='inspections',\
                           con=engine, if_exists='replace', chunksize=1000, index=True)

In [19]:
violations_df.to_sql(name='violations',\
                           con=engine, if_exists='replace', chunksize=1000, index=True)